# VI. Time series: visualizing

---
**Author(s):** Quentin Yeche, Kenji Ose, Dino Ienco - [UMR TETIS](https://umr-tetis.fr) / [INRAE](https://www.inrae.fr/)

---

## 1. Introduction

In this notebook we will introduce some tips for visualizing and masking time series.

## 2. Import libraries

As usual, we import all the required Python libraries.

In [ ]:
# STAC access
import pystac_client
import planetary_computer

# dataframes
import pandas as pd

# xarrays
import xarray as xr

# library for turning STAC objects into xarrays
import stackstac

# visualization
from matplotlib import pyplot as plt

# library for generating animated gif
import geogif

# miscellanous
import numpy as np
from IPython.display import display
from datetime import date
import time

## 3. Reloading of Sentinel-2 time series

Here we run the code of the [previous notebook](Joensuu_05-Time-series_part01.ipynb) in order to get the Sentinel-2 images time series, in the region of Montpellier. We add two arguments in the search:
- `sat:relative_orbit`: relative orbit
- `eo:cloud_cover`: cloud vover

In [ ]:
aoi_bounds = (3.875107329166124, 43.48641456618909, 
              4.118824575734205, 43.71739887308995)
cloud_nb = 20
# retrieving the relevant STAC Item
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
    )

today = date.today()
last_month = today.replace(month=today.month-1).strftime('%Y-%m')
time_range = f"2020-01-01/2021-01-01"#{last_month}"
search = catalog.search(
    collections=['sentinel-2-l2a'],
    datetime=time_range,
    bbox=aoi_bounds,
    query={"sat:relative_orbit": {"eq": 8}, 
           "eo:cloud_cover": {"lt": cloud_nb}},
    sortby="datetime"
)
items = search.item_collection()
print(f"{len(items)} items found")

time_steps_pc = len(items)

bands = ['B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B11', 'B12', 'SCL']
FILL_VALUE = 2**16-1
array = stackstac.stack(
                    items,
                    assets = bands,
                    resolution=10,
                    dtype="uint16",
                    fill_value=FILL_VALUE,
                    bounds_latlon=[3.944092,43.526638,4.014816,43.568420],#aoi_bounds,
                    chunksize= (time_steps_pc, 1, 'auto', 'auto')
                    )
array.drop_duplicates('time')
array 

## 4. Example of ploting time series images


### 4.1. Plotting by default

We plot the first 6 dates of the xarray in false-color composite.

In [ ]:
source = array.sel(band=["B08", "B04", "B03"])

rgb = source[:6]
starttime = time.time()
rgb.plot.imshow(col_wrap=3, col="time", rgb="band", vmax=2500, size=4)
print(f"time: {time.time()-starttime}")


We can also plot the NDVI...

In [ ]:
nir_src, red_src = source.sel(band="B08").astype('float'), source.sel(band="B04").astype('float')
ndvi_src = (nir_src - red_src) / (nir_src + red_src)
ndvi_plt = ndvi_src[:6]
ndvi_plt.plot.imshow(col_wrap=3, col="time", size=4, cmap='viridis')


As we can see on the previous plots, some images are cloudy. It's time to mask those invalid pixels.

### 4.2. Cloud masking

For masking the clouds, Sentinel-2 item contains an asset, called SCL, that allows masking different types of invalid pixels.

For information, SCL band is coded as follows:

|Bit| value Class             |
|---|-------------------------|
|0  | No data                 |
|1  | Saturated or defective  |
|2  | Dark area pixels        |
|3  | Cloud shadows           |
|4  | Vegetation              |
|5  | Bare Soil               |
|6  | Water                   |
|7  | Unclassified            |
|8  | Cloud medium probability|
|9  | Cloud high probability  |
|10 | Thin cirrus             |
|11 | Snow or ice             |
   
Values related to cloud and cloud shadows are kept here.

In [ ]:
# mask creation
SCL = array.sel(band = 'SCL')
mask = SCL.isin([3, 8, 9, 10])

# application of mask
result = array.where(~mask) #, 0) argument 2 = new output value for nan

source_m = result.sel(band=["B08", "B04", "B03"])
rgb = source_m[:6]
starttime = time.time()
rgb.plot.imshow(col_wrap=3, col="time", rgb="band", vmax=2500, size=4)
print(f"time: {time.time()-starttime}")

## 5. Gap Filling

We would like to perform a temporal **gapfilling** of our Sentinel-2 time series. We have to calculate an interpolated value for all masked (nodata) pixels. Here we choose the linear method and apply it to the masked xarray. Finally, we calculate again the NDVI.

In [ ]:
starttime = time.time()

interpolated = source_m.interpolate_na(dim="time", method="linear", use_coordinate = 'time')
interpolated = interpolated.bfill(dim= 'time')
interpolated.data = interpolated.data.astype(np.uint16)

rgb = interpolated[:6]

nir, red = interpolated.sel(band="B08").astype('float'), interpolated.sel(band="B04").astype('float')
ndvi_int = (nir - red) / (nir + red)

rgb.plot.imshow(col_wrap=3, col="time", rgb="band", vmax=2500, size=4)

print(f"time: {time.time()-starttime}")


Now, on one pixel, we compare the NDVI time series with and without gap-filling.

In [ ]:
x, y =562453, 4874880
ndvi_int_pt = ndvi_int.sel(x=x, y=y, method='nearest')
ndvi_src_pt = ndvi_src.sel(x=x, y=y, method='nearest')

plt.plot(ndvi_int_pt.time, ndvi_int_pt)
plt.plot(ndvi_src_pt.time, ndvi_src_pt)
plt.legend(['ndvi - interpolation', 'ndvi - source'])
plt.show()

## 6. Conversion of S2 time series into median time series

Reduce time dimension in function of quartile (4 images a year) and time with median.

In [ ]:
composites = interpolated.resample(time="M").median("time")
composites

In [ ]:
nir, red = composites.sel(band="B08").astype('float'), composites.sel(band="B04").astype('float')
ndvi_med = (nir - red) / (nir + red)

ndvi_med_pt = ndvi_med.sel(x=x, y=y, method='nearest')
plt.plot(ndvi_med_pt.time, ndvi_med_pt)
plt.show()

## 7. Extra: Make a nice-looking animation

### 7.1. Production of animated GIF

We use `GeoGIF` to turn the stack into an animation.

In [ ]:
gif_img = geogif.dgif(composites, fps=8).compute()
gif_img

If you want to export the animated image as a GIF file, you should use the following instructions. The file will be saved at the same location as the current notebook file. 

In [ ]:
gif_bytes = geogif.dgif(composites, fps=8, bytes=True, robust=True).compute()

with open("example.gif", "wb") as f:
    f.write(gif_bytes)
